# 🔍 IDE Crash 日志智能分析工具
> 基于 Ollama 本地大模型 + 知识注入，自动诊断 JetBrains IDE / DevEco Studio 崩溃根因

---

## 使用前准备
```bash
# 1. 安装并启动 Ollama
curl -fsSL https://ollama.com/install.sh | sh
ollama serve

# 2. 拉取模型（选一个）
ollama pull qwen2.5:7b        # 推荐，中文效果好
ollama pull nomic-embed-text  # RAG 模式专用嵌入模型

# 3. 安装 Python 依赖
pip install requests ipywidgets
```

---
## 三种使用模式
| 模式 | 适用场景 | 额外依赖 |
|------|---------|--------|
| **规则引擎** | 离线快速预判，无需 LLM | 无 |
| **System Prompt 直注** | 生产推荐，知识库 < 50 条 | 仅 requests |
| **RAG 检索增强** | 知识库大时扩展用 | nomic-embed-text |


---
## ① 环境初始化

In [2]:
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath('__file__')))

import json
import requests
import math
import textwrap
from IPython.display import display, Markdown, HTML, clear_output
import ipywidgets as widgets

from knowledge_base import KNOWLEDGE_RULES, SYSTEM_KNOWLEDGE_TEXT
from analyzer import (
    SystemPromptAnalyzer,
    RAGAnalyzer,
    RuleEnginePreFilter,
    list_models,
    chat,
    DEFAULT_MODEL,
    OLLAMA_BASE_URL,
)

print('✅ 模块加载成功')
print(f'   知识库规则数: {len(KNOWLEDGE_RULES)}')
print(f'   Ollama 地址 : {OLLAMA_BASE_URL}')

✅ 模块加载成功
   知识库规则数: 8
   Ollama 地址 : http://localhost:11434


In [3]:
# ── 检查 Ollama 连接状态 ──────────────────────────
try:
    models = list_models()
    display(Markdown('### ✅ Ollama 连接正常'))
    display(Markdown('**本地可用模型：**'))
    for m in models:
        print(f'  • {m}')
except Exception as e:
    display(Markdown(f'### ❌ 无法连接 Ollama\n> 请先执行 `ollama serve`\n\n错误：`{e}`'))

### ✅ Ollama 连接正常

**本地可用模型：**

  • qwen3:4b


In [4]:
# ── 全局配置（按需修改）──────────────────────────
CONFIG = {
    'model':       'qwen3:4b',     # 对话模型（改成你本地已 pull 的模型）
    'embed_model': 'nomic-embed-text', # 嵌入模型（RAG 模式用）
    'top_k':       3,                  # RAG 检索返回条数
    'temperature': 0.1,                # 低温度 = 更确定性的输出
}
print('当前配置：')
for k, v in CONFIG.items():
    print(f'  {k:15s} = {v}')

当前配置：
  model           = qwen3:4b
  embed_model     = nomic-embed-text
  top_k           = 3
  temperature     = 0.1


---
## ② 示例崩溃日志

In [7]:
# 内置示例日志（覆盖所有根因类型）
DEMO_LOGS = {
    'Mac JBR Metal崩溃': """
java.lang.IllegalStateException: Error - unable to initialize Metal after
recreation of graphics device. Cannot load metal library: No MTLDevice.
java.desktop/sun.awt.CGraphicsDevice.<init>(CGraphicsDevice.java:91)
Exception in NSApplicationAWT: java.lang.IllegalStateException: ...
""",
    'Windows虚拟内存不足': """
Native memory allocation (malloc) failed to allocate 1407664 bytes.
Error detail: Chunk::new
Out of Memory Error (arena.cpp:191), pid=2680, tid=9240
""",
    '物理内存不足': """
# Native memory allocation (malloc) failed to allocate 1330048 bytes.
# Error detail: Chunk::new
# Possible reasons:
#   The system is out of physical RAM or swap space
#   This process is running with CompressedOops enabled,
#   and the Java Heap may be blocking the growth of the native heap
""",
    'chrome_elf访问违例': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x0000000000000000, pid=928, tid=5776
# Problematic frame:
# C  [chrome_elf.dll+0x1b549]  java.lang.ProcessHandleImpl.getProcessPids0
""",
    'GC线程崩溃（疑似CPU）': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffd4c6c2580, pid=33548, tid=4488
# Problematic frame:
# V  [jvm.dll+0x3f6d67]
Current thread (0x000002617bfc3730): GCTaskThread "GC Thread#5"
[stack: 0x000000777e600000,0x000000777e700000] [id=22192]
""",
    'JBR-A-27偶发崩溃': """
# EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffcaed3c475, pid=17708, tid=5556
# JRE version: OpenJDK Runtime Environment JBR-17.0.12+1-1087.25-jcef (17.0.12+1)
# Java VM: OpenJDK 64-Bit Server VM JBR-17.0.12+1-1087.25-jcef
# Problematic frame:
# V  [jvm.dll+0x36c475]
""",
    'JBR空指针（backBuffers）': """
java.lang.NullPointerException: Cannot invoke
"java.awt.image.VolatileImage.getGraphics()"
because "this.backBuffers[i]" is null
""",
    'JDK Bug（字体渲染）': """
EXCEPTION_ACCESS_VIOLATION (0xc0000005) at pc=0x00007ffacb926d67, pid=21720, tid=12544
# Problematic frame:
# V  [jvm.dll+0x3f6d67]  sun.java2d.loops.DrawGlyphListLCD.DrawGlyphListLCD()
""",
}

print(f'共 {len(DEMO_LOGS)} 条示例日志：')
for i, name in enumerate(DEMO_LOGS, 1):
    print(f'  {i}. {name}')

共 8 条示例日志：
  1. Mac JBR Metal崩溃
  2. Windows虚拟内存不足
  3. 物理内存不足
  4. chrome_elf访问违例
  5. GC线程崩溃（疑似CPU）
  6. JBR-A-27偶发崩溃
  7. JBR空指针（backBuffers）
  8. JDK Bug（字体渲染）


---
## ③ 模式一：纯规则引擎（无需 LLM）
> 基于关键词匹配，离线可用，毫秒级响应

In [ ]:
engine = RuleEnginePreFilter()

def rule_engine_analyze(log_text: str) -> None:
    """规则引擎分析单条日志，带格式化输出"""
    result = engine.prefilter(log_text)
    if not result:
        display(Markdown('> ⚠️ **未命中任何规则** — 根因：未明确，需人工分析'))
        return

    rule = result['rule']
    conf_emoji = {'高': '🟢', '中': '🟡', '低': '🔴'}.get(result['confidence'], '⚪')

    md = f"""
### {conf_emoji} 根因：{rule['name']}

| 字段 | 内容 |
|------|------|
| **类别** | `{rule['category']}` |
| **置信度** | {result['confidence']}（命中 {result['matched_count']}/{result['total_keywords']} 个关键词）|
| **平台** | {', '.join(rule.get('platforms', ['全平台'])) or '全平台'} |

**📋 根因说明**
{rule['description']}

**🔧 处置建议**
"""
    for line in rule['solution'].strip().split('\n'):
        md += f'\n{line}'

    display(Markdown(md))

print('规则引擎已就绪 ✅')

In [ ]:
# ── 规则引擎：分析所有示例日志 ───────────────────
for name, log in DEMO_LOGS.items():
    display(Markdown(f'---\n#### 📄 日志：{name}'))
    rule_engine_analyze(log)

In [ ]:
# ── 规则引擎：分析自定义日志 ─────────────────────
my_log = """
# 在这里粘贴你的崩溃日志
Native memory allocation (mmap) failed to map 404750336 bytes.
Error detail: G1 virtual space
Out of Memory Error (os_windows.cpp:3825), pid=481984, tid=86464
"""

display(Markdown('#### 📄 自定义日志分析'))
rule_engine_analyze(my_log)

---
## ④ 模式二：System Prompt 直注法（推荐）
> 将完整诊断规则库注入 system 消息，LLM 具备完整推理能力

In [5]:
# 初始化分析器
sp_analyzer = SystemPromptAnalyzer(model=CONFIG['model'])

# 查看注入的 System Prompt 内容
display(Markdown(f'**System Prompt 长度：** {len(SYSTEM_KNOWLEDGE_TEXT)} 字符 (~{len(SYSTEM_KNOWLEDGE_TEXT)//4} tokens)'))
display(Markdown('**注入内容预览（前 400 字符）：**'))
print(SYSTEM_KNOWLEDGE_TEXT[:400] + '...')

[SystemPromptAnalyzer] 使用模型: qwen3:4b
[SystemPromptAnalyzer] 知识注入长度: 1744 字符


**System Prompt 长度：** 1744 字符 (~436 tokens)

**注入内容预览（前 400 字符）：**


你是一个专业的IDE崩溃日志分析专家，专注于JetBrains系列IDE和DevEco Studio的崩溃诊断。

## 诊断规则库

### 规则1：Mac外接显示器JBR崩溃
- 触发条件：出现 `IllegalStateException` + `No MTLDevice` + `Cannot load metal library`
- 根因：JBR在macOS上Metal图形设备重建失败
- 结论标签：JBR问题，mac上外接显示器导致JBR崩溃

### 规则2：JBR空指针（backBuffers）
- 触发条件：`NullPointerException` + `backBuffers[i] is null` + `VolatileImage.getGraphics()`
- 根因：JBR图形缓冲区未初始化
- 结论标签：JBR问题，空指针

### 规则3：Windows...


In [8]:
# ── 单条日志分析 ─────────────────────────────────
# 从下拉框选择示例，或直接修改 log_to_analyze
log_to_analyze = DEMO_LOGS['物理内存不足']

print('📝 待分析日志：')
print(log_to_analyze.strip())
print('\n' + '='*60)
print('🤖 LLM 分析结果（流式输出）：')
print('='*60 + '\n')

result = sp_analyzer.analyze(log_to_analyze, stream=True)

📝 待分析日志：
# Native memory allocation (malloc) failed to allocate 1330048 bytes.
# Error detail: Chunk::new
# Possible reasons:
#   The system is out of physical RAM or swap space
#   This process is running with CompressedOops enabled,
#   and the Java Heap may be blocking the growth of the native heap

🤖 LLM 分析结果（流式输出）：


[分析中] 模型: qwen3:4b

### 崩溃日志根因分析报告

**1. 根因类别**  
物理内存不足，内存分配失败导致crash（匹配规则4）

**2. 关键证据**  
```
Native memory allocation (malloc) failed to allocate 1330048 bytes.
The system is out of physical RAM or swap space
```

**3. 置信度**  
高（日志明确包含物理内存耗尽的直接诊断信息，且无歧义）

**4. 处置建议**  
1. **立即释放内存占用**：关闭非必要后台进程（如IDE的其他实例、浏览器、大型文件编辑器），优先释放IDE占用的内存。  
2. **增加系统交换空间**：在Windows中通过`系统属性 > 高级 > 虚拟内存`调整交换文件大小；在Linux中使用`swapon -s`检查并扩展swap分区。  
3. **运行内存健康检查**：使用Windows内置工具（`Windows Memory Diagnostic`）或Linux命令（`memtester`）验证物理内存硬件是否损坏。

**5. 未明确原因**  
不适用（置信度为高，日志明确指向物理内存耗尽，无模糊或未覆盖场景）  

---

### 诊断依据说明
- **为什么匹配规则4？**  
  日志中 `Native memory allocation (malloc) failed to allocate 1330048 

In [ ]:
# ── 批量分析（返回结构化 JSON）────────────────────
logs_list = list(DEMO_LOGS.values())
print(f'开始批量分析 {len(logs_list)} 条日志（stream=False 静默模式）...\n')

batch_results = sp_analyzer.batch_analyze(logs_list)

# 展示摘要表格
table_md = '| # | 日志预览 | 分析摘要（前200字） |\n|---|---------|-----------------|\n'
names = list(DEMO_LOGS.keys())
for r in batch_results:
    preview = r['crash_log'].replace('\n', ' ').strip()[:60]
    summary = r['analysis'].replace('\n', ' ').strip()[:200]
    table_md += f"| {r['index']} | `{preview}...` | {summary}... |\n"

display(Markdown(table_md))

In [ ]:
# ── 保存批量结果到 JSON ───────────────────────────
output_path = 'batch_analysis_results.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(batch_results, f, ensure_ascii=False, indent=2)
print(f'✅ 批量结果已保存至 {output_path}')

---
## ⑤ 模式三：RAG 检索增强法
> 向量检索 Top-K 最相关规则，再调用 LLM 生成分析报告
> 
> **前置条件**：`ollama pull nomic-embed-text`

In [ ]:
# ── 初始化 RAG（构建向量索引）────────────────────
try:
    rag_analyzer = RAGAnalyzer(
        chat_model=CONFIG['model'],
        embed_model=CONFIG['embed_model'],
        top_k=CONFIG['top_k'],
    )
    display(Markdown('✅ **RAG 索引构建完成**'))
except Exception as e:
    display(Markdown(
        f'❌ RAG 初始化失败：`{e}`\n\n'
        '> 请先执行：`ollama pull nomic-embed-text`'
    ))
    rag_analyzer = None

In [ ]:
# ── RAG 分析 + 查看检索分数 ──────────────────────
if rag_analyzer:
    test_log = DEMO_LOGS['Windows虚拟内存不足']

    print('📝 待分析日志：', test_log.strip()[:100], '...\n')

    detail = rag_analyzer.analyze_with_scores(test_log)

    # 展示检索命中的规则及分数
    display(Markdown('### 🔎 向量检索结果'))
    score_md = '| 排名 | 规则名称 | 相似度分数 |\n|------|---------|----------|\n'
    for i, r in enumerate(detail['retrieved_rules'], 1):
        bar = '█' * int(r['score'] * 20)
        score_md += f"| {i} | {r['name']} | {r['score']:.4f} `{bar}` |\n"
    display(Markdown(score_md))

    display(Markdown('### 🤖 LLM 分析结果'))
    display(Markdown(detail['analysis']))
else:
    print('RAG 分析器未初始化，跳过此单元格')

---
## ⑥ 混合模式（生产推荐）
> 规则引擎高置信度 → 直接返回（零 LLM 开销）  
> 规则引擎低置信度 → 自动升级到 LLM 深度分析

In [ ]:
def hybrid_analyze(log_text: str, model: str = CONFIG['model']) -> dict:
    """
    混合模式：规则引擎快速预判 + LLM 兜底
    返回: {'source': 'rule'|'llm', 'confidence': str, 'result': str}
    """
    pre = engine.prefilter(log_text)

    if pre and pre['confidence'] == '高':
        rule = pre['rule']
        return {
            'source': 'rule_engine',
            'confidence': '高',
            'category': rule['category'],
            'name': rule['name'],
            'description': rule['description'],
            'solution': rule['solution'],
            'llm_called': False,
        }
    else:
        # 规则引擎置信度不足，调用 LLM
        analyzer = SystemPromptAnalyzer(model=model)
        llm_result = analyzer.analyze(log_text, stream=False)
        return {
            'source': 'llm',
            'confidence': pre['confidence'] if pre else '低',
            'rule_hint': pre['rule']['name'] if pre else '无',
            'llm_analysis': llm_result,
            'llm_called': True,
        }


# 测试混合模式
display(Markdown('### 测试混合模式（各场景）'))
test_cases = {
    'Mac Metal崩溃（应命中规则引擎-高）': DEMO_LOGS['Mac JBR Metal崩溃'],
    '物理内存不足（规则引擎中-转LLM）':  DEMO_LOGS['物理内存不足'],
}

for case_name, log in test_cases.items():
    display(Markdown(f'#### 📄 {case_name}'))
    res = hybrid_analyze(log)

    if not res['llm_called']:
        display(Markdown(
            f'**⚡ 规则引擎直接返回（未调用LLM）**\n\n'
            f'- 根因类别：`{res["category"]}`\n'
            f'- 规则名称：`{res["name"]}`\n'
            f'- 说明：{res["description"]}\n'
            f'- 建议：\n{res["solution"]}'
        ))
    else:
        display(Markdown(
            f'**🤖 LLM 深度分析（规则引擎置信度不足）**\n\n'
            f'{res["llm_analysis"]}'
        ))

---
## ⑦ 交互式分析面板
> 通过 Widget 界面直接粘贴日志进行实时分析

In [ ]:
# ── Widget 交互面板 ───────────────────────────────
style = {'description_width': '120px'}
layout_full = widgets.Layout(width='100%')

# 控件定义
mode_select = widgets.ToggleButtons(
    options=[('🔑 规则引擎', 'rule'), ('🤖 LLM直注', 'system'), ('🔀 混合模式', 'hybrid')],
    value='hybrid',
    description='分析模式：',
    style=style,
)

example_select = widgets.Dropdown(
    options=[('📝 自定义输入', '')] + [(f'示例: {k}', v) for k, v in DEMO_LOGS.items()],
    value='',
    description='选择示例：',
    style=style,
    layout=layout_full,
)

log_input = widgets.Textarea(
    value='',
    placeholder='📋 在此粘贴崩溃日志（hs_err_pid*.log 内容）...',
    description='崩溃日志：',
    layout=widgets.Layout(width='100%', height='180px'),
    style=style,
)

analyze_btn = widgets.Button(
    description='  开始分析',
    button_style='primary',
    icon='search',
    layout=widgets.Layout(width='180px', height='40px'),
)

clear_btn = widgets.Button(
    description='清空',
    button_style='warning',
    icon='trash',
    layout=widgets.Layout(width='100px', height='40px'),
)

output_area = widgets.Output()

# 事件绑定
def on_example_change(change):
    if change['new']:
        log_input.value = change['new'].strip()

def on_clear(b):
    log_input.value = ''
    example_select.value = ''
    with output_area:
        clear_output()

def on_analyze(b):
    log_text = log_input.value.strip()
    if not log_text:
        with output_area:
            clear_output()
            display(Markdown('> ⚠️ 请先输入崩溃日志'))
        return

    mode = mode_select.value
    with output_area:
        clear_output(wait=True)
        display(Markdown(f'---\n### 分析结果（模式：{mode_select.label}）'))

        if mode == 'rule':
            rule_engine_analyze(log_text)

        elif mode == 'system':
            try:
                result = sp_analyzer.analyze(log_text, stream=False)
                display(Markdown(result))
            except Exception as e:
                display(Markdown(f'❌ LLM 调用失败：`{e}`'))

        elif mode == 'hybrid':
            try:
                res = hybrid_analyze(log_text)
                if not res['llm_called']:
                    display(Markdown(
                        f'**⚡ 规则引擎直接返回（未调用LLM）**\n\n'
                        f'- 根因：`{res["name"]}`\n'
                        f'- 说明：{res["description"]}\n'
                        f'- 建议：\n{res["solution"]}'
                    ))
                else:
                    display(Markdown(res['llm_analysis']))
            except Exception as e:
                display(Markdown(f'❌ 分析失败：`{e}`'))

example_select.observe(on_example_change, names='value')
analyze_btn.on_click(on_analyze)
clear_btn.on_click(on_clear)

# 渲染面板
display(HTML('<h3 style="margin-bottom:10px">🔍 IDE Crash 日志分析面板</h3>'))
display(widgets.VBox([
    mode_select,
    example_select,
    log_input,
    widgets.HBox([analyze_btn, clear_btn]),
    output_area,
]))

---
## ⑧ 批量分析文件夹中的日志文件

In [ ]:
import glob
from pathlib import Path

def analyze_log_folder(
    folder: str,
    pattern: str = 'hs_err_pid*.log',
    mode: str = 'hybrid',
    save_json: bool = True,
) -> list[dict]:
    """
    批量分析指定文件夹中所有崩溃日志文件。

    Args:
        folder:    日志文件夹路径
        pattern:   文件名匹配模式（默认匹配 hs_err_pid*.log）
        mode:      分析模式（'rule' / 'system' / 'hybrid'）
        save_json: 是否保存结果到 JSON 文件

    Returns:
        结构化分析结果列表
    """
    files = glob.glob(str(Path(folder) / pattern))
    if not files:
        print(f'⚠️  未找到匹配文件：{folder}/{pattern}')
        return []

    print(f'📂 找到 {len(files)} 个日志文件，开始分析...\n')
    results = []

    for i, filepath in enumerate(files, 1):
        fname = Path(filepath).name
        print(f'[{i}/{len(files)}] {fname}')

        with open(filepath, encoding='utf-8', errors='ignore') as f:
            log_text = f.read()

        # 提取关键错误段落（前 3000 字符通常包含核心报错）
        log_excerpt = log_text[:3000]

        if mode == 'rule':
            res = engine.prefilter(log_excerpt)
            entry = {
                'file': fname,
                'mode': 'rule',
                'category': res['rule']['category'] if res else '未明确',
                'name': res['rule']['name'] if res else '未明确',
                'confidence': res['confidence'] if res else '低',
            }
        else:  # system or hybrid
            if mode == 'hybrid':
                res = hybrid_analyze(log_excerpt)
                entry = {
                    'file': fname,
                    'mode': 'hybrid',
                    'llm_called': res.get('llm_called', False),
                    'analysis': res.get('llm_analysis') or res.get('description', ''),
                }
            else:
                analysis = sp_analyzer.analyze(log_excerpt, stream=False)
                entry = {'file': fname, 'mode': 'system', 'analysis': analysis}

        results.append(entry)
        print(f'  → {entry.get("name", entry.get("analysis", "")[:80])}')

    if save_json:
        out = Path(folder) / 'analysis_results.json'
        with open(out, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        print(f'\n✅ 结果已保存至 {out}')

    return results


# 使用示例（取消注释并修改路径）
# results = analyze_log_folder(
#     folder='/path/to/crash_logs/',
#     pattern='hs_err_pid*.log',
#     mode='hybrid',
# )

print('analyze_log_folder() 函数已定义，按需调用')

---
## ⑨ 扩展知识库（添加新规则）

In [ ]:
# ── 在运行时动态添加新规则（无需改文件）────────────
from knowledge_base import KNOWLEDGE_RULES

NEW_RULE = {
    'id': 'STACK_OVERFLOW',
    'category': 'JVM错误',
    'name': 'JVM栈溢出',
    'keywords': [
        'StackOverflowError',
        'stack overflow',
        'Stack: [',
        'free space=',
    ],
    'exception_types': ['StackOverflowError'],
    'negative_keywords': [],
    'platforms': [],
    'description': 'JVM线程调用栈深度超过限制，通常由无限递归或极深的调用链引起。',
    'solution': (
        '1. 排查代码中是否存在无限递归调用\n'
        '2. 增大线程栈大小：在 IDE 启动参数中添加 -Xss4m\n'
        '3. 检查递归终止条件是否正确'
    ),
}

# 动态注册新规则
existing_ids = {r['id'] for r in KNOWLEDGE_RULES}
if NEW_RULE['id'] not in existing_ids:
    KNOWLEDGE_RULES.append(NEW_RULE)
    # 重新初始化规则引擎以应用新规则
    engine = RuleEnginePreFilter()
    print(f'✅ 新规则已添加：{NEW_RULE["name"]}')
    print(f'   当前知识库规则数：{len(KNOWLEDGE_RULES)}')
else:
    print(f'⚠️  规则 {NEW_RULE["id"]} 已存在，跳过')

# 验证新规则生效
test_stack = 'java.lang.StackOverflowError\n\tat com.example.Foo.bar(Foo.java:10)\nStack: [0x00..., free space=8k'
res = engine.prefilter(test_stack)
print(f'\n验证新规则：{"✅ 命中" if res else "❌ 未命中"}')
if res:
    print(f'  → {res["rule"]["name"]}（置信度: {res["confidence"]}）')

---
## ⑩ 统计分析：批量日志根因分布

In [ ]:
from collections import Counter

# 对所有示例日志做规则引擎统计
all_logs = list(DEMO_LOGS.values())
categories = []

for log in all_logs:
    res = engine.prefilter(log)
    categories.append(res['rule']['category'] if res else '未明确')

counter = Counter(categories)
total = len(all_logs)

# 文本可视化（无需 matplotlib）
display(Markdown('### 📊 根因分布统计（示例日志集）'))
table = '| 根因类别 | 数量 | 占比 | 分布 |\n|---------|------|------|------|\n'
for cat, count in counter.most_common():
    pct = count / total * 100
    bar = '█' * int(pct / 5)
    table += f'| {cat} | {count} | {pct:.0f}% | `{bar}` |\n'

display(Markdown(table))
print(f'总计分析：{total} 条日志')